# Solvate and equilibrate a ligand in a box of water

In [ ]:
import time

import mdtraj
import nglview
import numpy
import openmm
import openmm.app
import openmm.unit
from openff.toolkit import ForceField, Molecule, unit
from openff.toolkit.typing.engines.smirnoff.forcefield import get_available_force_fields
from rich.pretty import pprint

from openff.interchange import Interchange
from openff.interchange.components._packmol import RHOMBIC_DODECAHEDRON, pack_box

## Construct the topology

In this example we'll construct a topology consisting of one ligand in a rhombic dodecahedral box with 2100 water molecules. We'll use a _mapped_ SMILES when creating `Molecule` objects to ensure the atom ordering matches. (Atom ordering is not strictly a part of SMILES and therefore liable to be changed with updates to RDKit.)

This can be extended or modified by i.e.
* Replacing this sample ligand with a different ligand of interest - substitute out the ligand SMILES
* Using a different number of water molecules - substitute out the `2100` used below
* Adding ions or co-solvents into the box - add more `Molecule` object as desired


In [ ]:
ligand = Molecule.from_mapped_smiles(
    "[H:7][C@:6]1([C:13](=[C:11]([C:9](=[O:10])[O:8]1)[O:12][H:19])[O:14][H:20])[C@:3]([H:4])([C:2]([H:16])([H:17])[O:1][H:15])[O:5][H:18]"
)
water = Molecule.from_mapped_smiles("[H:2][O:1][H:3]")

# Naming the residue is not needed to parameterize the system or run the simulation, but makes visualization easier
for atom in water.atoms:
    atom.metadata["residue_name"] = "HOH"

There are a few ways to convert the information in this trajectory to an Openff [`Topology`](https://docs.openforcefield.org/projects/toolkit/en/stable/api/generated/openff.toolkit.topology.Topology.html#openff.toolkit.topology.Topology) object. Since we already know how many of which molecules we want, we'll use a PACKMOL wrapper shipped with Interchange. The `Topology` object returned by `pack_box` contains the ligand, 1000 copies of water, the box vectors we asked for, and the positions generated by PACKMOL.

In [ ]:
topology = pack_box(
    molecules=[ligand, water],
    number_of_copies=[1, 1000],
    box_vectors=3.5 * RHOMBIC_DODECAHEDRON * unit.nanometer,
)
topology.n_molecules, topology.box_vectors, topology.get_positions().shape

The ["Sage"](https://openforcefield.org/community/news/general/sage2.0.0-release/) force field line (version 2.x.x) includes TIP3P  parameters for water, so we don't need to use multiple force fields to parametrize this topology. (One could use a different water model provided they accept the risks of using a different one than the force field was optimized with.)

Note that the "Parsley" (version 1.x.x) line did *not* include TIP3P parameters, so loading in an extra force field was required.

In [ ]:
sage = ForceField("openff-2.3.0.offxml")

From here, we can create an ``Interchange`` object, which stores the results of applying the force field to the topology. Since the `Topology` object contained positions and box vectors, we don't need to set them again - they're already set on the `Interchange` object!

In [ ]:
interchange: Interchange = Interchange.from_smirnoff(force_field=sage, topology=topology)
interchange.topology.n_atoms, interchange.box, interchange.positions.shape

Now, we can prepare everything that OpenMM needs to run and report a brief equilibration simulation:
* A [`Simulation`](http://docs.openmm.org/latest/api-python/generated/openmm.app.simulation.Simulation.html#openmm.app.simulation.Simulation) object containing
  * An `openmm.System`
  * A topology in OpenMM's object model (`openmm.app.Topology`)
  * Positions and box vectors in OpenMM's unit solution (`openmm.unit.Quantity`)
* A barostat, since we want to use NPT dynamics to relax the box size toward equilibrium
* An integrator
* Reporters for the trajectory and simulation data

For convenience, let's wrap some boilerplate code into a function that can be called again later with different inputs.

In [ ]:
def create_simulation(
    interchange: Interchange,
    pdb_stride: int = 500,
    trajectory_name: str = "trajectory.pdb",
) -> openmm.app.Simulation:
    integrator = openmm.LangevinMiddleIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        1 * openmm.unit.femtoseconds,
    )

    barostat = openmm.MonteCarloBarostat(
        1.0 * openmm.unit.bar,
        293.15 * openmm.unit.kelvin,
        25,
    )

    simulation = interchange.to_openmm_simulation(
        combine_nonbonded_forces=True,
        integrator=integrator,
        additional_forces=[barostat],
    )

    # https://github.com/openmm/openmm/issues/3736#issuecomment-1217250635
    simulation.minimizeEnergy()

    simulation.context.setVelocitiesToTemperature(300 * openmm.unit.kelvin)
    simulation.context.computeVirtualSites()

    pdb_reporter = openmm.app.PDBReporter(trajectory_name, pdb_stride)
    state_data_reporter = openmm.app.StateDataReporter(
        "data.csv",
        10,
        step=True,
        potentialEnergy=True,
        temperature=True,
        density=True,
    )
    simulation.reporters.append(pdb_reporter)
    simulation.reporters.append(state_data_reporter)

    return simulation

In [ ]:
simulation = create_simulation(interchange)

Finally, we can run this simulation. This should take approximately 10-20 seconds on a laptop or small workstation.

Again, let's wrap this up into a function to avoid copy-pasting code.

In [ ]:
def run_simulation(simulation: openmm.app.Simulation, n_steps: int = 5000):
    print("Starting simulation")
    start_time = time.process_time()

    print("Step, volume (nm^3)")

    for step in range(n_steps):
        simulation.step(1)
        if step % 500 == 0:
            box_vectors = simulation.context.getState().getPeriodicBoxVectors()
            print(step, numpy.linalg.det(box_vectors._value).round(3))

    end_time = time.process_time()
    print(f"Elapsed time: {(end_time - start_time):.2f} seconds")

We should observe (small) fluctuations in the density of the simulation, which is running under the NPT ensemble. If the initial density was a good guess, there shouldn't be significant changes from the initial value around $30 nm^3$.

In [ ]:
run_simulation(simulation)

## Appendix A: visualizing the trajectory

If [NGLView](http://nglviewer.org/nglview/latest/) is installed, we can use it and MDTraj to load and visualize the PDB trajectory:

In [ ]:
view = nglview.show_mdtraj(mdtraj.load("trajectory.pdb"))
view.add_representation("licorice", selection="water")
view

## Appendix B: Using the OPC water model

The `openff-forcefields` package now includes some common water models. After release 2023.05.1, this includes OPC, a 4-site water model from the Amber community. We can load it by simply passing it to the `ForceField` constructor after Sage. (When loading multiple force fields like this, parameters in each section are appended in order of the files.) We can inspect the virtual site parameter in our new in-memory force field.

In [ ]:
force_field_opc = ForceField("openff_unconstrained-2.3.0.offxml", "opc.offxml")
pprint(force_field_opc["VirtualSites"].to_dict())

We can also get a rough visualization of a single water molecule including the virtual site.

In [ ]:
water.generate_conformers(n_conformers=1)
view = force_field_opc.create_interchange(water.to_topology()).visualize(include_virtual_sites=True)
view.clear_representations()
view.add_representation("ball+stick", aspectRatio=1.5)
view

Since we want to use a different force field with the same chemical topology - a ligand in a box of water - we can re-use the same `Topology` object we prepared earlier, re-using the same functions we defined above!

In [ ]:
interchange_opc = Interchange.from_smirnoff(
    force_field=force_field_opc,
    topology=topology,
)

In [ ]:
interchange_opc.to_openmm()

In [ ]:
simulation = create_simulation(interchange_opc, trajectory_name="trajectory_opc.pdb")

run_simulation(simulation)

In [ ]:
def view_trajectory(
    trajectory: mdtraj.Trajectory,
    show_virtual_sites: bool = False,
) -> nglview.NGLWidget:
    if show_virtual_sites:
        view = nglview.show_mdtraj(trajectory)
    else:
        import numpy

        atom_indices = numpy.where([a.element.mass > 0.0 for a in trajectory[0].topology.atoms])[0]

        view = nglview.show_mdtraj(trajectory.atom_slice(atom_indices))

    view.add_representation("licorice", selection="water")
    return view


# NGLview likes to infer bonds between virtual sites in ways that look messy,
# but you can pass `show_virtual_sites=True` just to ensure they're there
view_trajectory(mdtraj.load("trajectory_opc.pdb"))

## Appendix C: Using the TIP4P-FB water model with GROMACS

The `openff-forcefields` package includes several other common water models. The toolkit provides an API for listing the force fields found in your current installation. We can filter out some force field lines and get a picture of the water models available to us. At the time of writing, TIP4P-FB and OPC3 are also provided, but future releases will likely add more. (See the [repo's documentation](https://github.com/openforcefield/openff-forcefields/blob/main/docs/water-models.md#versioning) for why multiple copies of each force field are provided with slightly different versions.)

In [ ]:
sorted(
    filter(
        lambda x: not x.startswith(("openff", "smirnoff99", "ff14sb")),
        get_available_force_fields(),
    )
)

To use a different force field, such as [TIP4P-FB](https://pubs.acs.org/doi/10.1021/jz500737m), simply point the `ForceField` constructor at a different file:

In [ ]:
sage_tip4p_fb = ForceField("openff-2.3.0.offxml", "tip4p_fb.offxml")

From here we can create a new `Interchange` object - only necessary since we're using a different force field - and use the `Interchange.to_gromacs` method to write GROMACS files. We'll also run an energy minimization before writing otu GROMACS files - this could also be done as an extra step with GROMACS, but it's convient to just call it from the Interchange API here.

In [ ]:
out = Interchange.from_smirnoff(
    force_field=sage_tip4p_fb,
    topology=topology,
)

# could take a few seconds
out.minimize()

out.to_gromacs("tip4p")

Now, with GROMACS files written, we can use the bundled `run.mdp` file to compile and run a GROMACS simulation!

In [ ]:
!gmx grompp -p tip4p.top -c tip4p.gro -f run.mdp -o run.tpr -maxwarn 1
!gmx mdrun --deffnm run

In [ ]:
# This view might show erroneous bonds across the periodic boundaries, but individual waters should look fine
view_trajectory(mdtraj.load("run.xtc", top="run.gro"))